# **Install Jira module beneath**

In [1]:
pip install jira

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.5/77.5 kB 392.3 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 1.6 MB/s eta 0:00:00


In [ ]:
# @title Login
from jira import JIRA
import pandas as pd
from datetime import datetime

Account_Name =  'name' 
Password = 'password'  
jira = JIRA('https://aaa.com/jira/', auth=(Account_Name,Password))

INFO:jira:Trying to refresh the cookie auth session...


In [ ]:
# @title Basic_Demand
issue_keys = "BUG2IN1-14402,BUG2IN1-14639,BUG2IN1-14621" 
new_issue_keys = issue_keys.split(',')

issue_data_list = []

for issue_key in new_issue_keys:
    try:
        issue = jira.issue(issue_key)
        issue_data = {
            'IssueKey': issue.key,
            'Summary': issue.fields.summary,
            'Status': issue.fields.status.name,
            'Updated': datetime.strptime(issue.fields.updated[:10], "%Y-%m-%d"),
            'Assignee': issue.fields.assignee.displayName,
        }
        issue_data_list.append(issue_data)
    except Exception as e:
        print(f"問題 {issue_key} 不存在。錯誤信息: {e}")
        issue_data_list.append({
            'IssueKey': issue_key,
            'Summary': None,
            'Status': None,
            'Updated': None,
            'Assignee': None,
        })
report_df1 = pd.DataFrame(issue_data_list)
Document_name = "technical_bi-weekly_demo" 
report_df1.to_excel(Document_name + '.xlsx', index=False)

問題  不存在。錯誤信息: JiraError HTTP 405 url: https://ec-service.asus.com/jira/rest/api/2/issue/
	
	response headers = {'Date': 'Thu, 13 Jun 2024 03:40:40 GMT', 'Content-Type': 'text/html;charset=UTF-8', 'Connection': 'keep-alive', 'X-DataDome': 'protected', 'Accept-CH': 'Sec-CH-UA,Sec-CH-UA-Mobile,Sec-CH-UA-Platform,Sec-CH-UA-Arch,Sec-CH-UA-Full-Version-List,Sec-CH-UA-Model,Sec-CH-Device-Memory', 'Set-Cookie': 'datadome=kKFgYPuSA3iQj7xbSRG6RMXo40XyNB5LymPEispoMrg_e46o1OHy16m3Ws~r75PFkxeKBBvATJDdo5gA1RDU225ZCpS4IYVFbUPxZ9pLw0bDOuO5jmzS3DJe5t7kwZHW; Max-Age=31536000; Domain=.asus.com; Path=/; Secure; SameSite=Lax, TS01ad4907=0188058bf9b03d3a2bf9e092c931662ee9d7807c3cf7e3dd1834562815a7fd403ae6b5c307322d3dc1090d9f3ee94fc82f7d77e11e; Path=/; Secure; HTTPOnly, TS01d862b6=0188058bf9b03d3a2bf9e092c931662ee9d7807c3cf7e3dd1834562815a7fd403ae6b5c307322d3dc1090d9f3ee94fc82f7d77e11e; path=/; domain=.asus.com; HTTPonly; Secure', 'X-AREQUESTID': '700x33579168x1', 'X-ASESSIONID': '6tg4pj', 'Referrer-Policy':

In [ ]:
# @title Basic_Demand + Comment
issue_keys = "BUG2IN1-14402,BUG2IN1-14639,BUG2IN1-14621" 
new_issue_keys = issue_keys.split(',')

issue_data_list = []

for issue_key in new_issue_keys:
    latest_feedback = {}

    try:
        issue = jira.issue(issue_key)
        comments = jira.issue(issue_key).fields.comment.comments
        latest_comment = None
        if comments:
            latest_comment = comments[-1]
        if latest_comment:
            comment_body = latest_comment.body
            comment_time = latest_comment.created
        else:
            comment_body = None
            comment_time = None
        issue_data = {
            'IssueKey': issue.key,
            'Summary': issue.fields.summary,
            'Status': issue.fields.status.name,
            'Updated': datetime.strptime(issue.fields.updated[:10], "%Y-%m-%d"),
            'Assignee': issue.fields.assignee.displayName,
            'Comment_content': comment_body,
            'Comment_time': comment_time,
        }

        issue_data_list.append(issue_data)
    except Exception as e:
        print(f"問題 {issue_key} 不存在。錯誤信息: {e}")
        issue_data_list.append({
            'IssueKey': issue_key,
            'Summary': None,
            'Status': None,
            'Updated': None,
            'Assignee': None,
            'Comment_content': None,
            'Comment_time': None,
        })

report_df2 = pd.DataFrame(issue_data_list)
Document_name = "technical_bi-weekly_C_0606"
report_df2.to_excel(Document_name + '.xlsx', index=False)

In [ ]:
# @title Basic_Demand + Related Issue

issue_keys = "BUG2IN1-14402,BUG2IN1-14639,BUG2IN1-14621,BUG2IN1-14646"
new_issue_keys = issue_keys.split(',')

issue_data_list = []

for issue_key in new_issue_keys:
    try:
        issue = jira.issue(issue_key)

        related_issues = [] 

        issue_data = {
            'IssueKey': issue.key,
            'Summary': issue.fields.summary,
            'Status': issue.fields.status.name,
            'Updated': datetime.strptime(issue.fields.updated[:10], "%Y-%m-%d"),
        }

        assignee = issue.fields.assignee
        if assignee:
            issue_data['Assignee'] = assignee.displayName
        else:
            issue_data['Assignee'] = None

        issuelinks = issue.fields.issuelinks if hasattr(issue.fields, 'issuelinks') else []
        for link in issuelinks:
            if hasattr(link, 'inwardIssue') and hasattr(link.inwardIssue, 'key'):
                inward_key = link.inwardIssue.key
                inward_issue = jira.issue(inward_key)
                related_data = {
                    'IssueKey': issue.key,
                    'Summary': issue.fields.summary,
                    'Status': issue.fields.status.name,
                    'Updated': datetime.strptime(issue.fields.updated[:10], "%Y-%m-%d"),
                    'Assignee': issue.fields.assignee,
                    'RelatedIssueKey': inward_key if inward_key else None,
                    'RelatedIssueSummary': inward_issue.fields.summary if inward_issue.fields.summary else None,
                    'RelatedIssueStatus': inward_issue.fields.status.name if inward_issue.fields.status else None,
                    'RelatedIssueDueDate': datetime.strptime(inward_issue.fields.duedate[:10], "%Y-%m-%d") if inward_issue.fields.duedate else None,
                    'LinkedIssuetype': link.type.name if link.type.name else None,
                }
                related_issues.append(related_data)

            if hasattr(link, 'outwardIssue') and hasattr(link.outwardIssue, 'key'):
                outward_key = link.outwardIssue.key
                outward_issue = jira.issue(outward_key)
                related_data = {
                    'IssueKey': issue.key,
                    'Summary': issue.fields.summary,
                    'Status': issue.fields.status.name,
                    'Updated': datetime.strptime(issue.fields.updated[:10], "%Y-%m-%d"),
                    'Assignee': issue.fields.assignee,
                    'RelatedIssueKey': outward_key if outward_key else None,
                    'RelatedIssueSummary': outward_issue.fields.summary if outward_issue.fields.summary else None,
                    'RelatedIssueStatus': outward_issue.fields.status.name if outward_issue.fields.status else None,
                    'RelatedIssueDueDate': datetime.strptime(outward_issue.fields.duedate[:10], "%Y-%m-%d") if outward_issue.fields.duedate else None,
                    'LinkedIssuetype': link.type.name if link.type.name else None,
                }
                related_issues.append(related_data)

        issue_data_list.append(issue_data)

        issue_data_list.extend(related_issues)

    except Exception as e:
        print(f"問題 {issue_key} 不存在。錯誤訊息: {e}")
        issue_data_list.append({
            'IssueKey': issue_key,
            'Summary': None,
            'Status': None,
            'Updated': None,
            'Assignee': None,
            'RelatedIssueKey': None,
            'RelatedIssueSummary': None,
            'RelatedIssueStatus': None,
            'RelatedIssueDueDate': None,
        })

report_df3 = pd.DataFrame(issue_data_list)
report_df3 = report_df3[~(report_df3.duplicated(subset=['IssueKey'], keep=False) & report_df3['RelatedIssueKey'].isna())]
report_df3.drop_duplicates(subset=['IssueKey', 'RelatedIssueKey'], keep='first', inplace=True)

Document_name = "technical_bi-weekly_R_0606"
report_df3.to_excel(Document_name + '.xlsx', index=False)